In [6]:
import tensorflow as tf
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


In [12]:
#loading the dataset and unpacking the training and testing data along with the dataset info
(ds_train , ds_test) , ds_info = tfds.load('oxford_iiit_pet' , split = ['train' , 'test'] , with_info = True)

#checking the dataset info
print(type(ds_train),type(ds_test))
print(ds_info.features['label'].num_classes) 

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'> <class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
37


In [14]:
#printing the class names and the number of classes
print(ds_info.features['label'].names[:5])
print(len(ds_info.features['label'].names))

['Abyssinian', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle']
37


In [13]:
#split sizes
print(ds_info.splits['train'].num_examples)
print(ds_info.splits['test'].num_examples)

3680
3669


In [16]:
# 90% of the original train as new train, 10% as validation
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'oxford_iiit_pet',
    split=['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True,
    with_info=True
)

'train[:90%]' → takes the first 90% of the original training set for training.

'train[90%:]' → takes the remaining 10% as a validation set.

'test' → keeps the official test split provided by the dataset.

as_supervised=True → gives  simple (image, label) pairs.

with_info=True → returns extra info (class names, counts, etc.) in ds_info.


ds_train: training dataset

ds_val: validation dataset

ds_test: test dataset. 

ds_info: metadata (number of classes, feature info, etc.)

In [ ]:
IMG_SIZE = 128     #resize the images to 128x128 for faster training
BATCH_SIZE = 32  #number of images to process in each training step
AUTOTUNE = tf.data.AUTOTUNE  #automatically tune the prefetching for optimal performance and lets TensorFlow optimize the data loading and preprocessing pipeline for better performance.

#preprocessing function to resize the images and normalize the pixel values
def preprocess_train(image , label):
    image = tf.image.resize(image , (IMG_SIZE , IMG_SIZE)) #resize the image to 128x128
    Image = tf.cast(image , tf.float32) /255.0 #normalize the pixel values to be between 0 and 1
    image = tf.image.random_flip_left_right(image) # data augmentation by randomly flipping the image horizontally
    return image ,  label     #return the preprocessed image and its corresponding label

def preprocess_eval(image , label):
    image = tf.image.resize(image , (IMG_SIZE , IMG_SIZE)) #resize the image to